## Computer Vision with Convolutional Neural Networks (CNNs)
### A Hands-On Keras Exercise (using CIFAR-10 Dataset)

**Objective:** This assignment will introduce you to Convolutional Neural Networks (CNNs) for image classification using the CIFAR-10 dataset. You will learn to load and preprocess color image datasets, build and train CNNs, apply regularization techniques (Batch Normalization, Dropout), experiment with optimizers, and evaluate model performance.

**Estimated Time:** 30-45 Minutes

**Materials:**

* Jupyter Notebook environment (or Google Colab)
* Python 3 installed with `tensorflow` (which includes Keras), `scikit-learn`, `matplotlib`, and `numpy`.

---

### Part 1: Setting the Stage - Libraries and Data Loading

**1.1 Load Essential Python Libraries**

* **Task:** Begin by importing all the necessary libraries for this assignment. Pay attention to the new layers specifically for CNNs.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Keras for building neural networks
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical # For one-hot encoding labels

# Specific CNN layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

# Scikit-learn for data splitting and metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

**Relevant Documentation:**
* `Conv2D`: This is the convolutional layer for 2D inputs (like images). It applies a set of learnable filters to the input, creating feature maps. [Keras Conv2D Documentation](https://keras.io/api/layers/convolution_layers/convolution2d/)
    * **Sample Syntax:** `layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3))`
* `MaxPooling2D`: This is a pooling layer that reduces the spatial dimensions (height and width) of the input. It helps in reducing computational cost and extracting dominant features. [Keras MaxPooling2D Documentation](https://keras.io/api/layers/pooling_layers/max_pooling2d/)
    * **Sample Syntax:** `layers.MaxPooling2D((2, 2))`
* `Flatten`: This layer flattens the input from a multi-dimensional shape (e.g., from convolutional layers) into a 1D array. This is necessary before passing the data to fully connected (`Dense`) layers. [Keras Flatten Documentation](https://keras.io/api/layers/reshaping_layers/flatten/)
    * **Sample Syntax:** `layers.Flatten()`

**1.2 Load a Computer Vision Dataset (CIFAR-10)**

* **Task:** Load the CIFAR-10 dataset, which consists of 60,000 32x32 color images in 10 classes, with 6,000 images per class.
* **Instructions:**
    * Use `keras.datasets.cifar10.load_data()` to load the dataset. It comes pre-split into training and testing sets.
    * Print the shapes of `X_train`, `y_train`, `X_test`, and `y_test` to understand their dimensions.

In [ ]:
# Load dataset using variable names  X_train_full, y_train_full, X_test, y_test







print(f"X_train_full shape: {X_train_full.shape}") # (num_samples, height, width, channels)
print(f"y_train_full shape: {y_train_full.shape}") # (num_samples, 1)
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

num_classes = len(np.unique(y_train_full))
print(f"Number of classes: {num_classes}")

# Define class names for CIFAR-10 for better interpretation of results
cifar10_class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
                       'dog', 'frog', 'horse', 'ship', 'truck']

**1.3 Data Exploration: Visualize Sample Images**

* **Task:** Display a few sample images from the training set to get a feel for the data.
* **Instructions:**
    * Use `matplotlib.pyplot` to plot 5-10 images.
    * Display the corresponding label for each image using the `cifar10_class_names`.

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(10):
    plt.subplot(2, 5, i + 1)
    plt.imshow(X_train_full[i]) # CIFAR-10 images are color, no cmap needed
    plt.title(f"Label: {cifar10_class_names[y_train_full[i][0]]}") # y_train_full is 2D array
    plt.axis('off')
plt.tight_layout()
plt.show()

---

### Part 2: Data Preprocessing for CNNs

**2.1 Reshape Data for CNN Input (No Reshaping Needed for CIFAR-10)**

* **Task:** CIFAR-10 images are already in the `(batch_size, height, width, channels)` format, with 3 channels for RGB. So, no explicit reshaping is needed here, but we will define `input_shape`.
* **Instructions:** Define the `input_shape` based on the loaded data dimensions.

In [ ]:
img_rows, img_cols, channels = X_train_full.shape[1], X_train_full.shape[2], X_train_full.shape[3]
input_shape = (img_rows, img_cols, channels) # 3 for RGB channels

print(f"Input shape for CNN: {input_shape}")
print(f"X_train_full shape (already correct): {X_train_full.shape}")
print(f"X_test shape (already correct): {X_test.shape}")

**Relevant Documentation:**
The channel dimension is crucial for CNNs, especially for color images. It represents the different color channels (e.g., Red, Green, Blue for RGB images). CNNs process information across these channels to identify features based on color variations, unlike grayscale images which have only one channel. For CIFAR-10, images are 32x32 pixels with 3 color channels, so the input shape is (32, 32, 3).

**2.2 Normalize Pixel Values**

* **Task:** Scale the pixel values from the 0-255 range to a 0-1 range. This helps neural networks train more effectively.
* **Instructions:** Convert the data type to float and divide by 255.

In [ ]:
# Normalize the dataset using variable names X_train_normalized,X_test_normalized



print(f"Min pixel value (train): {np.min(X_train_normalized)}")
print(f"Max pixel value (train): {np.max(X_train_normalized)}")

**Relevant Documentation:**
Normalizing pixel values (scaling them to a 0-1 range) is beneficial for neural networks because:
* **Faster Convergence:** It helps gradient descent-based optimizers converge more quickly as all features are on a similar scale.
* **Improved Performance:** Prevents larger input values from dominating the learning process, leading to more stable and potentially better model performance.
* **Activation Function Range:** Many activation functions (like sigmoid or tanh) work best with inputs close to zero. Normalization ensures inputs fall within these optimal ranges.
    * **Sample Syntax:** `X_data = X_data.astype('float32') / 255`

**2.3 One-Hot Encode Labels**

* **Task:** Convert integer labels (0-9) into a one-hot encoded format (e.g., `5` becomes `[0,0,0,0,0,1,0,0,0,0]`). This is necessary for `categorical_crossentropy` loss.
* **Instructions:** Use `tf.keras.utils.to_categorical`.

In [ ]:
# try using one hot encoding using variable names y_train_one_hot,y_test_one_hot

print(f"Original label for first sample: {y_train_full[0][0]}") # y_train_full is 2D array
print(f"One-hot encoded label for first sample: {y_train_one_hot[0]}")
print(f"y_train_one_hot shape: {y_train_one_hot.shape}")

**Relevant Documentation:**
* `sparse_categorical_crossentropy`: This loss function is used when your labels are integers (e.g., 0, 1, 2 for classes). It performs an implicit one-hot encoding internally.
* `categorical_crossentropy`: This loss function is used when your labels are already in one-hot encoded format (e.g., `[0,0,1,0]` for class 2). You must explicitly convert your integer labels to one-hot encoding using `to_categorical`.

Choose `categorical_crossentropy` when your output layer uses `softmax` activation and your labels are one-hot encoded. If your labels are integers and your output is `softmax`, `sparse_categorical_crossentropy` is more convenient. [Keras Losses Documentation](https://keras.io/api/losses/)
    * **Sample Syntax:** `y_one_hot = to_categorical(y_integer_labels, num_classes=10)`

**2.4 Data Splitting (Train, Validation, Test)**

* **Task:** Split your training data into a smaller training set and a validation set. The test set is already separate.
* **Instructions:** Use `train_test_split` on the *normalized and one-hot encoded* training data (`X_train_normalized`, `y_train_one_hot`) for your training and validation sets. Aim for an 80/20 or 90/10 split.

In [ ]:
# Split data set. you can use variable names X_train, X_val, y_train, y_val

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test_normalized.shape}, y_test shape: {y_test_one_hot.shape}")

**Relevant Documentation:**
Having separate training, validation, and test sets is fundamental for robust model evaluation:
* **Training Set:** Used to train the model and adjust its weights.
* **Validation Set:** Used to tune hyperparameters (like learning rate, number of layers, dropout rates) and monitor for overfitting during training. It provides an unbiased evaluation of a model fit on the training dataset while tuning model hyperparameters.
* **Test Set:** Used for a final, unbiased evaluation of the model's performance after all hyperparameter tuning is complete. This set should *never* be used during training or hyperparameter tuning to ensure the model generalizes well to new, unseen data.

If you only use a train-test split and tune hyperparameters based on the test set, you risk overfitting to the test set, leading to an overly optimistic estimate of your model's real-world performance.
    * **Sample Syntax:** `X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)`

---

### Part 3: Building and Training Your First CNN Model

**3.1 Create a Simple CNN Model**

* **Task:** Construct a basic CNN model. This will involve `Conv2D`, `MaxPooling2D`, `Flatten`, and `Dense` layers.
* **Instructions:**
    * Define a `Sequential` model.
    * Add a `Conv2D` layer (e.g., 32 filters, (3,3) kernel, 'relu' activation, `input_shape` for the first layer).
    * Add a `MaxPooling2D` layer (e.g., (2,2) pool size).
    * Add another `Conv2D` layer and `MaxPooling2D` layer (optional, but good for deeper features).
    * Add a `Flatten` layer to convert the 2D feature maps into a 1D vector.
    * Add one or more `Dense` layers for classification (e.g., 128 neurons, 'relu' activation).
    * Add the final `Dense` output layer with `num_classes` neurons and `softmax` activation.
    * Print `model.summary()`.

In [ ]:
# Create network model







**Relevant Documentation:**
* **`Conv2D` layers:** These layers apply convolutional filters to the input image. Each filter learns to detect specific features (e.g., edges, textures, patterns). By sliding these filters across the image, they create feature maps that highlight where these features are present. This process allows the network to learn hierarchical representations of the image.
* **`MaxPooling2D` layers:** These layers perform downsampling by taking the maximum value within a specified window (pool size). This reduces the spatial dimensions of the feature maps, which helps in:
    * Reducing the number of parameters and computational complexity.
    * Making the model more robust to small shifts or distortions in the input image (translational invariance).
These layers work together to progressively extract more complex and abstract features from the raw pixel data.
    * **Sample Syntax:**
        ```python
        model = Sequential([
            Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
            MaxPooling2D((2, 2))
        ])
        ```

**3.2 Compile the Model**

* **Task:** Configure the learning process by specifying the optimizer, loss function, and metrics.
* **Instructions:**
    * Use `model.compile()`.
    * For one-hot encoded labels, use `loss='categorical_crossentropy'`.
    * Start with the `Adam` optimizer.
    * Include `metrics=['accuracy']`.

In [ ]:
# compile the model







**Relevant Documentation:**
The **learning rate** is a crucial hyperparameter that determines the step size at each iteration while moving toward a minimum of a loss function. A learning rate that is too small can lead to slow convergence, while a learning rate that is too large can cause the optimization to overshoot the minimum or even diverge. It significantly impacts how quickly and effectively a model learns.
    * **Sample Syntax:** `model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])`

**3.3 Train the Model**

* **Task:** Train your CNN using the training and validation data.
* **Instructions:**
    * Use `model.fit()`.
    * Pass `X_train`, `y_train` as training data.
    * Pass `X_val`, `y_val` as validation data (using `validation_data` argument).
    * Set `epochs` (e.g., 10-20, as CNNs can take longer) and `batch_size` (e.g., 32 or 64).
    * Store the training history object in a variable (e.g., `history_simple_cnn`).

In [ ]:
# adjust below code
history_simple_cnn = model_simple_cnn.fit(X_train, y_train,
                                          epochs=15,
                                          batch_size=64,
                                          validation_data=(X_val, y_val),
                                          verbose=1)

**Relevant Documentation:**
* **Epochs:** An epoch represents one complete pass through the entire training dataset. In each epoch, the model sees every training example once. More epochs generally lead to better learning, but too many can cause overfitting.
* **Batch Size:** The batch size defines the number of training examples utilized in one iteration. Smaller batch sizes introduce more noise into the gradient but can help escape local minima. Larger batch sizes provide a more accurate estimate of the gradient but require more memory.

**Signs of Overfitting/Underfitting:**
* **Underfitting:** If both training and validation loss are high, and both accuracies are low, the model is likely underfitting. It hasn't learned enough from the training data.
* **Overfitting:** If training loss continues to decrease and training accuracy continues to increase, but validation loss starts increasing and validation accuracy plateaus or decreases, the model is overfitting. It's memorizing the training data instead of generalizing.
    * **Sample Syntax:** `history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))`

---

### Part 4: Model Improvement Techniques and Visualization

**4.1 Visualize Training History (Simple CNN)**

* **Task:** Plot the training and validation loss, and accuracy over epochs for your simple CNN. This is crucial for performance analysis.
* **Instructions:**
    * Access the `history_simple_cnn.history` dictionary.
    * Plot 'accuracy' vs. 'val_accuracy' and 'loss' vs. 'val_loss' using `matplotlib.pyplot`.
    * Add titles, labels, and legends.

In [ ]:
def plot_training_history(history, title_prefix="Model"):
    # Plot training & validation accuracy values
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title(f'{title_prefix} Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title(f'{title_prefix} Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.tight_layout()
    plt.show()

plot_training_history(history_simple_cnn, title_prefix="Simple CNN")

**Relevant Documentation:**
Analyzing the training and validation curves is key to diagnosing model performance:
* **Overfitting:** If the training accuracy continues to increase while validation accuracy plateaus or decreases, and training loss continues to decrease while validation loss increases, the model is overfitting. It is learning the training data too well, including its noise, and failing to generalize to new data.
* **Underfitting:** If both training and validation accuracy are low and both losses are high, the model is underfitting. It is too simple or hasn't been trained long enough to capture the underlying patterns in the data.
* **Good Fit:** Ideally, both training and validation curves should converge and remain close to each other, indicating that the model is learning effectively and generalizing well.

**4.2 Incorporate Batch Normalization and Dropout**

* **Task:** Create a new CNN model that includes `BatchNormalization` and `Dropout` layers. These are crucial for preventing overfitting in deeper networks.
* **Instructions:**
    * Define a *new* `Sequential` model.
    * **Placement Strategy:** A common pattern is `Conv2D` -> `BatchNormalization` -> `Activation` -> `Dropout` -> `MaxPooling2D`.
    * Add `BatchNormalization()` after each `Conv2D` layer (before or after activation, common to put before for ReLU).
    * Add `Dropout()` layers after `MaxPooling2D` or after `Dense` layers, with a `rate` (e.g., 0.25 for convolutional blocks, 0.5 for dense layers).
    * Compile and train this new model. Store its history (`history_improved_cnn`).

In [ ]:
# Create new model and use improvement techniques described above.
model_improved_cnn = Sequential([



])

model_improved_cnn.summary()

model_improved_cnn.compile(optimizer='adam',
                           loss='categorical_crossentropy',
                           metrics=['accuracy'])

print("\n--- Training Improved CNN Model ---")
history_improved_cnn = model_improved_cnn.fit(X_train, y_train,
                                               epochs=20, # More epochs might be needed
                                               batch_size=64,
                                               validation_data=(X_val, y_val),
                                               verbose=1)

plot_training_history(history_improved_cnn, title_prefix="Improved CNN (Adam)")

**Relevant Documentation:**
* **Batch Normalization:** This technique normalizes the activations of the previous layer at each batch, meaning it centers and scales the inputs to the next layer. This helps in:
    * **Stabilizing Learning:** Reduces internal covariate shift, allowing each layer to learn more independently.
    * **Faster Convergence:** Allows for higher learning rates.
    * **Regularization:** Adds a slight regularization effect, reducing the need for other regularization techniques.
    [Keras BatchNormalization Documentation](https://keras.io/api/layers/normalization_layers/batch_normalization/)
    * **Sample Syntax:** `layers.BatchNormalization()`

* **Dropout:** During training, Dropout randomly sets a fraction of input units to 0 at each update step. This prevents neurons from co-adapting too much and forces the network to learn more robust features. It acts as a powerful regularization technique to prevent overfitting.
    [Keras Dropout Documentation](https://keras.io/api/layers/regularization_layers/dropout/)
    * **Sample Syntax:** `layers.Dropout(0.5)`

By adding these layers, you should observe that the gap between training and validation curves narrows, and validation accuracy might improve or remain stable for longer, indicating better generalization and reduced overfitting compared to the simple CNN.

**4.3 Experiment with Different Optimizers on Improved Model**

* **Task:** Re-train your *improved* model using different optimizers (e.g., `SGD`, `RMSprop`) and compare their performance, especially focusing on convergence speed and final accuracy.
* **Instructions:**
    * For each optimizer:
        * Create a *fresh instance* of `model_improved_cnn` (important to reset weights).
        * Compile it with the new optimizer (e.g., `optimizer=SGD(learning_rate=0.01)`).
        * Train it for the same number of epochs and batch size.
        * Store the history object.
    * Plot the validation accuracy/loss for all three optimizers (Adam, SGD, RMSprop) on the *same graph* for comparison.

In [ ]:
def create_fresh_improved_cnn():
    # Helper function to create a new instance of the improved CNN architecture
    return Sequential([
        # complete the code with your own improved model










    ])

# Experiment with SGD
print("\n--- Training with SGD Optimizer ---")
model_sgd_improved = create_fresh_improved_cnn()
model_sgd_improved.compile(optimizer=SGD(learning_rate=0.01),
                          loss='categorical_crossentropy',
                          metrics=['accuracy'])
history_sgd_improved = model_sgd_improved.fit(X_train, y_train,
                                              epochs=20,
                                              batch_size=64,
                                              validation_data=(X_val, y_val),
                                              verbose=0) # Set verbose to 0 for cleaner output

# Experiment with RMSprop
print("\n--- Training with RMSprop Optimizer ---")
model_rmsprop_improved = create_fresh_improved_cnn()
model_rmsprop_improved.compile(optimizer=RMSprop(learning_rate=0.001),
                              loss='categorical_crossentropy',
                              metrics=['accuracy'])
history_rmsprop_improved = model_rmsprop_improved.fit(X_train, y_train,
                                                      epochs=20,
                                                      batch_size=64,
                                                      validation_data=(X_val, y_val),
                                                      verbose=0)

# Plotting comparison of Validation Accuracy
plt.figure(figsize=(10, 6))
plt.plot(history_improved_cnn.history['val_accuracy'], label='Adam')
plt.plot(history_sgd_improved.history['val_accuracy'], label='SGD')
plt.plot(history_rmsprop_improved.history['val_accuracy'], label='RMSprop')
plt.title('Validation Accuracy Comparison (Improved CNN)')
plt.xlabel('Epoch')
plt.ylabel('Validation Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# Plotting comparison of Validation Loss
plt.figure(figsize=(10, 6))
plt.plot(history_improved_cnn.history['val_loss'], label='Adam')
plt.plot(history_sgd_improved.history['val_loss'], label='SGD')
plt.plot(history_rmsprop_improved.history['val_loss'], label='RMSprop')
plt.title('Validation Loss Comparison (Improved CNN)')
plt.xlabel('Epoch')
plt.ylabel('Validation Loss')
plt.legend()
plt.grid(True)
plt.show()

**Relevant Documentation:**
Optimizers are algorithms or methods used to change the attributes of your neural network such as weights and learning rate in order to reduce the losses. Different optimizers have different approaches to navigating the loss landscape:
* **Adam (Adaptive Moment Estimation):** Combines ideas from RMSprop and momentum. It calculates adaptive learning rates for each parameter. Generally performs well across a wide range of problems and is often a good default choice.
* **SGD (Stochastic Gradient Descent):** The simplest optimizer, it updates weights in the direction of the negative gradient of the loss function. Can be slow to converge and prone to oscillations, but with momentum, it can be very effective.
    * **Sample Syntax:** `optimizer=SGD(learning_rate=0.01)`
* **RMSprop (Root Mean Square Propagation):** Adapts the learning rate for each parameter based on the magnitudes of recent gradients. It's good for recurrent neural networks and can handle non-stationary objectives.
    * **Sample Syntax:** `optimizer=RMSprop(learning_rate=0.001)`

You might observe differences in:
* **Convergence Speed:** How quickly the validation loss decreases and accuracy increases.
* **Stability:** How smooth the training curves are (less oscillation).
* **Final Performance:** The best validation accuracy achieved.

[Keras Optimizers Documentation](https://keras.io/api/optimizers/)

---

### Part 5: Model Evaluation on Test Set

**5.1 Evaluate Final Model**

* **Task:** Evaluate the performance of your *best-performing* model (from your optimizer experiments in Part 4.3) on the unseen test set.
* **Instructions:**
    * Choose the model that showed the best validation accuracy (e.g., `model_improved_cnn` if Adam was best).
    * Use `model.evaluate()` on `X_test_normalized` and `y_test_one_hot`.
    * Print the test loss and test accuracy.

In [ ]:
# Assuming model_improved_cnn (trained with Adam) was your best performing model
# If SGD or RMSprop was better, use model_sgd_improved or model_rmsprop_improved
best_model = model_improved_cnn # Change this if a different optimizer performed better

test_loss, test_accuracy = best_model.evaluate(X_test_normalized, y_test_one_hot, verbose=0)
print(f"\nTest Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

**Relevant Documentation:**
The test accuracy provides a final, unbiased estimate of your model's performance on unseen data. Comparing it to the validation accuracy helps assess **generalization**:
* If test accuracy is significantly lower than validation accuracy, it might indicate that your model overfit the validation set during hyperparameter tuning, or that the validation set was not perfectly representative of the true data distribution.
* If test accuracy is similar to validation accuracy, it suggests that your model generalizes well to new data and your validation set was a good proxy for unseen data.

**5.2 Make Predictions and Analyze Metrics**

* **Task:** Make predictions on the test set and calculate additional classification metrics.
* **Instructions:**
    * Use `best_model.predict()` on `X_test_normalized` to get probability predictions.
    * Convert probabilities to class predictions (e.g., use `np.argmax`).
    * Convert `y_test_one_hot` back to integer labels for `sklearn.metrics` functions.
    * Calculate and print the `accuracy_score`, `confusion_matrix`, and `classification_report` from `sklearn.metrics`.

In [ ]:
y_pred_probs = best_model.predict(X_test_normalized)
y_pred = np.argmax(y_pred_probs, axis=1) # Convert probabilities to class labels
y_test_labels = np.argmax(y_test_one_hot, axis=1) # Convert one-hot to integer labels

print("\n--- Classification Metrics on Test Set ---")
print(f"Accuracy Score: {accuracy_score(y_test_labels, y_pred):.4f}")
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred, target_names=cifar10_class_names))

**Relevant Documentation:**
* **Confusion Matrix:** A table that summarizes the performance of a classification model. Each row represents the instances in an actual class, while each column represents the instances in a predicted class. It helps visualize the types of errors made by the classifier (e.g., false positives, false negatives).
    [Scikit-learn Confusion Matrix Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html)
    * **Sample Syntax:** `confusion_matrix(y_true, y_pred)`

* **Classification Report:** Provides a more detailed breakdown of performance per class, including:
    * **Precision:** The ability of the classifier not to label as positive a sample that is negative. (True Positives / (True Positives + False Positives))
    * **Recall:** The ability of the classifier to find all the positive samples. (True Positives / (True Positives + False Negatives))
    * **F1-Score:** The harmonic mean of precision and recall. A good balance between precision and recall.
    * **Support:** The number of actual occurrences of the class in the specified dataset.
    [Scikit-learn Classification Report Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)
    * **Sample Syntax:** `classification_report(y_true, y_pred, target_names=['class_0', 'class_1'])`

Examining the confusion matrix and classification report will help identify which classes your model performs well on and which it struggles with. For example, high false positives for one class and high false negatives for another might indicate confusion between those specific categories (e.g., 'cat' and 'dog' in CIFAR-10).

---

### Challenge & Extension Activities (If Time Permits)

* **Data Augmentation:** Implement basic data augmentation (e.g., rotation, shifting, zooming) using `ImageDataGenerator` from `tf.keras.preprocessing.image` to improve generalization.
* **Deeper Network:** Create a CNN with more `Conv2D` and `MaxPooling2D` layers. Be mindful of vanishing gradients or overfitting.
* **Different Datasets:** Try classifying `fashion_mnist` (images of clothing).
* **Callbacks:** Add Keras Callbacks like `EarlyStopping` to stop training when validation performance plateaus, or `ModelCheckpoint` to save the best model during training.
* **Learning Rate Schedule:** Experiment with different learning rate schedules (e.g., reducing learning rate on plateau).
* **Visualize Feature Maps:** Try to visualize the output of intermediate `Conv2D` layers to understand what features the CNN is learning.

---

### Deliverables:

Students should submit their Jupyter Notebook containing all the code, outputs, and answers to the discussion questions.